In [1]:
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
def create_custom_cnn():
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [4]:
def create_transfer_learning_model():
  vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
  for layer in vgg16_model.layers:
    layer.trainable = False
  x = vgg16_model.output
  x = Flatten()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(10, activation='softmax')(x)
  model = Model(inputs=vgg16_model.input, outputs=predictions)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [5]:
custom_cnn_model = create_custom_cnn()
transfer_learning_model = create_transfer_learning_model()

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
custom_cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1563/1563 [==============================] - 14s 5ms/step - loss: 1.5197 - accuracy: 0.4438 - val_loss: 1.2345 - val_accuracy: 0.5659
Epoch 2/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1241 - accuracy: 0.6027 - val_loss: 1.0622 - val_accuracy: 0.6294
Epoch 3/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9671 - accuracy: 0.6628 - val_loss: 0.9309 - val_accuracy: 0.6758
Epoch 4/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8641 - accuracy: 0.7005 - val_loss: 0.8833 - val_accuracy: 0.6948
Epoch 5/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.7875 - accuracy: 0.7260 - val_loss: 0.9025 - val_accuracy: 0.6922
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.7271 - accuracy: 0.7457 - val_loss: 0.9305 - val_accuracy: 0.6800
Epoch 7/10
1563/1563 [==============================] - 9s 5ms/step - loss: 0.6756 - accuracy: 0.7646 - val_loss: 0.8971 - val_accuracy

In [7]:
transfer_learning_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1563/1563 [==============================] - 18s 11ms/step - loss: 1.3166 - accuracy: 0.5367 - val_loss: 1.2167 - val_accuracy: 0.5686
Epoch 2/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1431 - accuracy: 0.5988 - val_loss: 1.1466 - val_accuracy: 0.6019
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.0663 - accuracy: 0.6270 - val_loss: 1.1673 - val_accuracy: 0.5912
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.0055 - accuracy: 0.6436 - val_loss: 1.1325 - val_accuracy: 0.6045
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.9496 - accuracy: 0.6650 - val_loss: 1.1100 - val_accuracy: 0.6166
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.8942 - accuracy: 0.6845 - val_loss: 1.1299 - val_accuracy: 0.6143
Epoch 7/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.8466 - accuracy: 0.6996 - val_loss: 1.0970 -

In [8]:
custom_cnn_loss, custom_cnn_accuracy = custom_cnn_model.evaluate(X_test, y_test)
transfer_learning_loss, transfer_learning_accuracy = transfer_learning_model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 8ms/step - loss: 1.1680 - accuracy: 0.6225


In [9]:
print("CNN from Scratch Accuracy:", custom_cnn_accuracy)
print("Transfer Learning Accuracy:", transfer_learning_accuracy)

CNN from Scratch Accuracy: 0.7098000049591064
Transfer Learning Accuracy: 0.6225000023841858


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
                            rotation_range=15,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            fill_mode='nearest'
                            )

In [11]:
custom_cnn_model = create_custom_cnn()
transfer_learning_model = create_transfer_learning_model()

In [12]:
custom_cnn_model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))
transfer_learning_model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1563/1563 [==============================] - 35s 21ms/step - loss: 1.6310 - accuracy: 0.3985 - val_loss: 1.2876 - val_accuracy: 0.5288
Epoch 2/10
1563/1563 [==============================] - 32s 20ms/step - loss: 1.2931 - accuracy: 0.5364 - val_loss: 1.1989 - val_accuracy: 0.5740
Epoch 3/10
1563/1563 [==============================] - 33s 21ms/step - loss: 1.1671 - accuracy: 0.5881 - val_loss: 1.0166 - val_accuracy: 0.6493
Epoch 4/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.0815 - accuracy: 0.6195 - val_loss: 0.9963 - val_accuracy: 0.6505
Epoch 5/10
1563/1563 [==============================] - 33s 21ms/step - loss: 1.0275 - accuracy: 0.6396 - val_loss: 0.9544 - val_accuracy: 0.6775
Epoch 6/10
1563/1563 [==============================] - 31s 20ms/step - loss: 0.9813 - accuracy: 0.6535 - val_loss: 0.8625 - val_accuracy: 0.6998
Epoch 7/10
1563/1563 [==============================] - 32s 20ms/step - loss: 0.9406 - accuracy: 0.6726 - val_loss: 0.9738 -

In [14]:
cnn_from_scratch_loss_new, cnn_from_scratch_acc_new = custom_cnn_model.evaluate( X_test, y_test)
transfer_learning_loss_new, transfer_learning_acc_new = transfer_learning_model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 8ms/step - loss: 1.1129 - accuracy: 0.6146


In [15]:
print("CNN from Scratch Accuracy (without Augmentation):", custom_cnn_accuracy)
print("CNN from Scratch Accuracy (with Augmentation):", cnn_from_scratch_acc_new)
print("Transfer Learning Accuracy (without Augmentation):", transfer_learning_accuracy)
print("Transfer Learning Accuracy (with Augmentation):", transfer_learning_acc_new)

CNN from Scratch Accuracy (without Augmentation): 0.7098000049591064
CNN from Scratch Accuracy (with Augmentation): 0.7236999869346619
Transfer Learning Accuracy (without Augmentation): 0.6225000023841858
Transfer Learning Accuracy (with Augmentation): 0.6146000027656555
